<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

# <center>Using Arize with AI agents</center>

This guide shows you how to create and evaluate agents with Arize to improve performance. We'll go through the following steps:

* Create a customer support agent using a router template

* Trace the agent activity, including function calling

* Create a dataset to benchmark performance

* Evaluate agent performance using code, human annotation, and LLM as a judge

* Experiment with different prompts and models

# Initial setup


We'll setup our libraries, keys, and OpenAI tracing using Phoenix.

### Install Libraries

In [ ]:
!pip install -qq "arize-otel>0.6.0" "openinference-instrumentation-openai>=0.1.0" "arize-phoenix-evals>=0.17.5" "arize[Datasets]>7.29.0"

!pip install -qq openai==1.57.1 opentelemetry-sdk==1.28.2 opentelemetry-exporter-otlp==1.28.2 gcsfs==2024.10.0 nest_asyncio==1.6.0

### Setup Keys

In [ ]:
import os
from getpass import getpass
import nest_asyncio

nest_asyncio.apply()

SPACE_ID = globals().get("SPACE_ID") or getpass(
    "🔑 Enter your Arize Space ID: "
)
API_KEY = globals().get("API_KEY") or getpass("🔑 Enter your Arize API Key: ")
OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
    "🔑 Enter your OpenAI API key: "
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

To follow with this tutorial, you'll need to sign up for Arize and get your API key. You can see the [guide here](https://docs.arize.com/arize/llm-tracing/quickstart-llm).

In [ ]:
# Import open-telemetry dependencies
from arize.otel import register

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="agents-cookbook",  # name this to whatever you would like
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.openai import OpenAIInstrumentor

# Finish automatic instrumentation
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

# Create customer support agent

We'll be creating a customer support agent using function calling following the architecture below:

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/agent_architecture.png" width="800"/>

### Setup functions and create customer support agent

We have 6 functions that we define below.

1. product_comparison
2. product_search
3. customer_support
4. track_package
5. product_details
6. apply_discount_code



In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "product_comparison",
            "description": "Compare features of two products.",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_a_id": {
                        "type": "string",
                        "description": "The unique identifier of Product A.",
                    },
                    "product_b_id": {
                        "type": "string",
                        "description": "The unique identifier of Product B.",
                    },
                },
                "required": ["product_a_id", "product_b_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "product_search",
            "description": "Search for products based on criteria.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query string.",
                    },
                    "category": {
                        "type": "string",
                        "description": "The category to filter the search.",
                    },
                    "min_price": {
                        "type": "number",
                        "description": "The minimum price of the products to search.",
                        "default": 0,
                    },
                    "max_price": {
                        "type": "number",
                        "description": "The maximum price of the products to search.",
                    },
                    "page": {
                        "type": "integer",
                        "description": "The page number for pagination.",
                        "default": 1,
                    },
                    "page_size": {
                        "type": "integer",
                        "description": "The number of results per page.",
                        "default": 20,
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "customer_support",
            "description": "Get contact information for customer support regarding an issue.",
            "parameters": {
                "type": "object",
                "properties": {
                    "issue_type": {
                        "type": "string",
                        "description": "The type of issue (e.g., billing, technical support).",
                    }
                },
                "required": ["issue_type"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "track_package",
            "description": "Track the status of a package based on the tracking number.",
            "parameters": {
                "type": "object",
                "properties": {
                    "tracking_number": {
                        "type": "integer",
                        "description": "The tracking number of the package.",
                    }
                },
                "required": ["tracking_number"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "product_details",
            "description": "Returns details for a given product id",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_id": {
                        "type": "string",
                        "description": "The id of a product to look up.",
                    }
                },
                "required": ["product_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "apply_discount_code",
            "description": "Applies the discount code to a given order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "integer",
                        "description": "The id of the order to apply the discount code to.",
                    },
                    "discount_code": {
                        "type": "string",
                        "description": "The discount code to apply",
                    },
                },
                "required": ["order_id", "discount_code"],
            },
        },
    },
]

We define a function below called run_prompt, which uses the chat completion call from OpenAI with functions

In [ ]:
import os

import openai

client = openai.Client()


def run_prompt(input):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        tools=tools,
        tool_choice="auto",
        messages=[
            {
                "role": "system",
                "content": " ",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )

    if (
        hasattr(response.choices[0].message, "tool_calls")
        and response.choices[0].message.tool_calls is not None
        and len(response.choices[0].message.tool_calls) > 0
    ):
        tool_calls = response.choices[0].message.tool_calls
    else:
        tool_calls = []

    if response.choices[0].message.content is None:
        response.choices[0].message.content = ""
    if response.choices[0].message.content:
        return response.choices[0].message.content
    else:
        return tool_calls

Let's test it and see if it returns the right function! Based on whether we set tool_choice to "auto" or "required", the router will have different behavior.

In [ ]:
run_prompt("Hi, I'd like to apply to apply a discount code to my order.")

Now we have a basic agent, let's generate a dataset of questions and run the prompt against this dataset!

# Create synthetic dataset of questions

Using the template below, we're going to generate a dataframe of 25 questions we can use to test our customer support agent.

In [ ]:
GEN_TEMPLATE = """
You are an assistant that generates complex customer service questions.
The questions should often involve:

Multiple Categories: Questions that could logically fall into more than one category (e.g., combining product details with a discount code).
Vague Details: Questions with limited or vague information that require clarification to categorize correctly.
Mixed Intentions: Queries where the customer’s goal or need is unclear or seems to conflict within the question itself.
Indirect Language: Use of indirect or polite phrasing that obscures the direct need or request (e.g., using "I was wondering if..." or "Perhaps you could help me with...").
For specific categories:

Track Package: Include vague timing references (e.g., "recently" or "a while ago") instead of specific dates.
Product Comparison and Product Search: Include generic descriptors without specific product names or IDs (e.g., "high-end smartphones" or "energy-efficient appliances").
Apply Discount Code: Include questions about discounts that might apply to hypothetical or past situations, or without mentioning if they have made a purchase.
Product Details: Ask for comparisons or details that involve multiple products or categories ambiguously (e.g., "Tell me about your range of electronics that are good for home office setups").

Examples of More Challenging Questions
"There's an issue with one of the items I think I bought last month—what should I do?"
"I need help with something I ordered, or maybe I'm just looking for something new. Can you help?"

Some questions should be straightforward uses of the provided functions

Respond with a list, one question per line. Do not include any numbering at the beginning of each line. Do not include any category headings.
Generate 25 questions. Be sure there are no duplicate questions.
"""

In [ ]:
import nest_asyncio
import pandas as pd

nest_asyncio.apply()
from phoenix.evals import OpenAIModel

pd.set_option("display.max_colwidth", 500)

model = OpenAIModel(model="gpt-4o", max_tokens=1300)

In [ ]:
resp = model(GEN_TEMPLATE)

In [ ]:
split_response = resp.strip().split("\n")

questions_df = pd.DataFrame(split_response, columns=["question"])
print(questions_df)

Now let's use this dataset and run it against the router prompt above!

In [ ]:
response_df = questions_df.copy(deep=True)
response_df["response"] = response_df["question"].apply(run_prompt)
response_df["response"] = response_df["response"].astype(str)

In [ ]:
response_df

# Evaluating your agent

Now that we have a set of test cases, we can create evaluators to measure performance. This way, we don't have to manually inspect every single trace to see if the LLM is doing the right thing.

Here, we are defining our evaluation templates to judge whether the router selected a function correctly, whether it selected the right function, and whether it filled the arguments correctly.

In [ ]:
ROUTER_EVAL_TEMPLATE = """ You are comparing a response to a question, and verifying whether that response should have made a function call instead of responding directly. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [LLM Response]: {response}
    ************
    [END DATA]

Compare the Question above to the response. You must determine whether the reponse
decided to call the correct function.
Your response must be single word, either "correct" or "incorrect",
and should not contain any text or characters aside from that word.
"incorrect" means that the agent should have made function call instead of responding directly and did not, or the function call chosen was the incorrect one.
"correct" means the selected function would correctly and fully answer the user's question.

Here is more information on each function:
product_comparison: Compare features of two products. Should include either the product id or name. If the name or id is present in the question and not present in the generated function, the response is incorrect.
product_search: Search for products based on criteria.
track_package: Track the status of a package based on the tracking number.
customer_support: Get contact information for customer support regarding an issue. The response should always include an email or phone number.
apply_discount_code: Applies a discount code to an order.
product_details: Get detailed features on one product.
"""

FUNCTION_SELECTION_EVAL_TEMPLATE = """You are comparing a function call response to a question and trying to determine if the generated call is correct. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [LLM Response]: {response}
    ************
    [END DATA]

Compare the Question above to the function call. You must determine whether the function call
will return the answer to the Question. Please focus on whether the very specific
question can be answered by the function call.
Your response must be single word, either "correct", "incorrect", or "not-applicable",
and should not contain any text or characters aside from that word.
"incorrect" means that the function call will not provide an answer to the Question.
"correct" means the function call will definitely provide an answer to the Question.
"not-applicable" means that response was not a function call.

Here is more information on each function:
product_comparison: Compare features of two products. Should include either the product id or name. If the name or id is present in the question and not present in the generated function, the response is incorrect.
product_search: Search for products based on criteria.
track_package: Track the status of a package based on the tracking number.
customer_support: Get contact information for customer support regarding an issue. The response should always include an email or phone number.
apply_discount_code: Applies a discount code to an order.
product_details: Get detailed features on one product.
"""

PARAMETER_EXTRACTION_EVAL_TEMPLATE = """ You are comparing a function call response to a question and trying to determine if the generated call has extracted the exact right parameters from the question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [LLM Response]: {response}
    ************
    [END DATA]

Compare the parameters in the generated function against the JSON provided below.
The parameters extracted from the question must match the JSON below exactly.
Your response must be single word, either "correct", "incorrect", or "not-applicable",
and should not contain any text or characters aside from that word.
"incorrect" means that the parameters in the function do not match the JSON schema below exactly, or the generated function does not correctly answer the user's question.
You should also respond with "incorrect" if the response makes up information that is not in the JSON schema.
"correct" means the function call parameters match the JSON below and provides only relevant information.
"not-applicable" means that response was not a function call.

Here is more information on each function:
product_comparison: Compare features of two products. Should include either the product id or name. If the name or id is present in the question and not present in the generated function, the response is incorrect.
product_search: Search for products based on criteria.
track_package: Track the status of a package based on the tracking number.
customer_support: Get contact information for customer support regarding an issue. The response should always include an email or phone number.
apply_discount_code: Applies a discount code to an order.
product_details: Get detailed features on one product.
"""

Let's run evaluations using Phoenix's llm_classify function for our responses dataframe we generated above!

In [ ]:
from phoenix.evals import OpenAIModel, llm_classify

rails = ["incorrect", "correct"]

router_eval_df = llm_classify(
    dataframe=response_df,
    template=ROUTER_EVAL_TEMPLATE,
    model=OpenAIModel(model="gpt-4o"),
    rails=rails,
    provide_explanation=True,
    include_prompt=True,
    concurrency=4,
)

function_selection_eval_df = llm_classify(
    dataframe=response_df,
    template=FUNCTION_SELECTION_EVAL_TEMPLATE,
    model=OpenAIModel(model="gpt-4o"),
    rails=rails,
    provide_explanation=True,
    include_prompt=True,
    concurrency=4,
)

parameter_extraction_eval_df = llm_classify(
    dataframe=response_df,
    template=PARAMETER_EXTRACTION_EVAL_TEMPLATE,
    model=OpenAIModel(model="gpt-4o"),
    rails=rails,
    provide_explanation=True,
    include_prompt=True,
    concurrency=4,
)

Let's look at and inspect the results of our evaluatiion!

In [ ]:
router_eval_df

In [ ]:
function_selection_eval_df

In [ ]:
parameter_extraction_eval_df

# Create an experiment

With our dataset of questions we generated above, we can use our experiments feature to track changes across models, prompts, parameters for our agent.

Let's create this dataset and upload it into the platform.

In [ ]:
from arize.experimental.datasets import ArizeDatasetsClient
from uuid import uuid1
from arize.experimental.datasets.experiments.types import (
    ExperimentTaskResultColumnNames,
    EvaluationResultColumnNames,
)
from arize.experimental.datasets.utils.constants import GENERATIVE

# Your developer key, space id and api key are used from above
developer_key = getpass("🔑 Enter your Arize Developer key: ")

# Set up the arize client
arize_client = ArizeDatasetsClient(developer_key=developer_key, api_key=API_KEY)

dataset_name = "agents-cookbook-" + str(uuid1())[:5]

dataset_id = arize_client.create_dataset(
    space_id=SPACE_ID,
    dataset_name=dataset_name,
    dataset_type=GENERATIVE,
    data=questions_df,
)
dataset = arize_client.get_dataset(space_id=SPACE_ID, dataset_id=dataset_id)
print(dataset)

In [ ]:
# Map the evaluation results to the dataset
response_df["example_id"] = dataset["id"]

response_df["router_eval_label"] = router_eval_df["label"]
response_df["router_eval_explanation"] = router_eval_df["explanation"]
response_df["parameter_eval_label"] = parameter_extraction_eval_df["label"]
response_df["parameter_eval_explanation"] = parameter_extraction_eval_df[
    "explanation"
]
response_df["function_eval_label"] = function_selection_eval_df["label"]
response_df["function_eval_explanation"] = function_selection_eval_df[
    "explanation"
]

# Define column mappings for task
task_cols = ExperimentTaskResultColumnNames(
    example_id="example_id", result="response"
)
# Define column mappings for evaluator
router_evaluator_cols = EvaluationResultColumnNames(
    label="router_eval_label",
    explanation="router_eval_explanation",
)
parameter_evaluator_cols = EvaluationResultColumnNames(
    label="parameter_eval_label",
    explanation="parameter_eval_explanation",
)
function_evaluator_cols = EvaluationResultColumnNames(
    label="function_eval_label",
    explanation="function_eval_explanation",
)

In [ ]:
# Use with ArizeDatasetsClient.log_experiment()
arize_client.log_experiment(
    space_id=SPACE_ID,
    experiment_name="my_experiment" + str(uuid1())[:5],
    experiment_df=response_df,
    task_columns=task_cols,
    evaluator_columns={
        "router": router_evaluator_cols,
        "parameter_extraction": parameter_evaluator_cols,
        "function_selection": function_evaluator_cols,
    },
    dataset_name=dataset_name,
)